# Examples of usage:

In [1]:
!pip install -r requirements.txt

In [2]:
from tool_box import tool_box
import pandas as pd
import os
from pandas import DataFrame

###  DICOM to nrrd conversion

In [3]:
##Convert DICOM dataset to the volume (nrrd) format
# export_path = ... export path where the converted nrrds will be placed, region_of_interest = ... if you know exact name of
# the ROI you want to extract then write it with the ! character infront, eg. region_of_interest = !gtv1 , 
# if you want to extract all the gtv's in the rtstructure eg. gtv1, gtv2, gtv_whatever then just specify the stem word 
# eg. region_of_interest = gtv, default value is region_of_interest ='all', where all ROI's in rtstruct will be extracted.
        

In [4]:
# Set up the parameters for the 
parameters = {'data_path': r'./data/dcms/', #path_to_your_data
              'data_type': 'dcm',
              'multi_rts_per_pat': False}   #When False, it will look only for 1 rtstruct in the patient folder, this will
                                            #speed up the process, if you have more then 1 rtstruct per patient, set it to True
                  
export_path =r'./data/'    

In [5]:
#Inintialize the dataset 
Data_MRI = tool_box(**parameters) 

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.81it/s]


In [6]:
#Run the conversion
Data_MRI.convert_to_nrrd(export_path,'gtv')

Patients converted: 100%|████████████████████████████████████████████████████████████████| 3/3 [00:08<00:00,  2.79s/it]


### Quick check of the ROI's in the nrrd dataset

In [7]:
#If you want to check your converted images&ROIs or just any volumetric dataset (nrrd/mha), you can use get_jpegs function of
#the toolbox, it will generate jpeg images for each slice with the image and overlap of the contour.

In [8]:
Data_MRI_nrrd = tool_box(data_path = r'./data/converted_nrrds/',
                    data_type='nrrd')

100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1498.50it/s]


In [9]:
Data_MRI_nrrd.get_jpegs(r'./data/')

Patients processed: 100%|████████████████████████████████████████████████████████████████| 5/5 [01:02<00:00, 12.55s/it]


In [10]:
#Lets just check one of the patient's right in jupyter

In [11]:
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import interact
import numpy as np
from PIL import Image

def browse_images(images,names):
    n = len(images)
    def view_image(i):
        plt.figure(figsize=(20,10))
        plt.imshow(images[i])#, cmap=plt.cm.gray_r, interpolation='nearest')
        plt.title('Slice: %s' % names[i])
        plt.axis('off')
        plt.show()
    interact(view_image, i=(0,n-1))
    
for pat,_ in Data_MRI_nrrd:
    _,file_struct = [*os.walk(os.path.join('./data/images_quick_check/',pat))]
    root,images = file_struct[0],file_struct[2]
    imgs =[np.array(Image.open(os.path.join(root,img))) for img in images]
    print(pat)
    browse_images(imgs,images)
    break


    


LUNG1-001_20180209_CT_2_GTV-1_mask


interactive(children=(IntSlider(value=9, description='i', max=19), Output()), _dom_classes=('widget-interact',…

### Extract PyRadiomics features

In [12]:
Data_MRI_nrrd = tool_box(data_path = r'./data/converted_nrrds/',
                    data_type='nrrd')

100%|████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<?, ?it/s]


In [13]:
#Use your own pyradiomics parameters file, more you can find here: https://github.com/Radiomics/pyradiomics 
parameters = r"./example_ct_parameters.yaml"
features = Data_MRI_nrrd.extract_features(parameters,loggenabled=True)

Patients processed: 100%|████████████████████████████████████████████████████████████████| 5/5 [01:14<00:00, 14.89s/it]


In [14]:
features.head(5)

,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,...,wavelet-LLL_gldm_HighGrayLevelEmphasis,wavelet-LLL_gldm_LargeDependenceEmphasis,wavelet-LLL_gldm_LargeDependenceHighGrayLevelEmphasis,wavelet-LLL_gldm_LargeDependenceLowGrayLevelEmphasis,wavelet-LLL_gldm_LowGrayLevelEmphasis,wavelet-LLL_gldm_SmallDependenceEmphasis,wavelet-LLL_gldm_SmallDependenceHighGrayLevelEmphasis,wavelet-LLL_gldm_SmallDependenceLowGrayLevelEmphasis,Patient,ROI
0,v3.0,1.18.3,1.1.0,0.5.2,3.6.4,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [1.0, 2.0, 3...",01ae3d400f3f6fdacc121a85f52fd9dba8be2253,3D,"(0.9765625, 0.9765625, 3.0)",...,14462.536758039314,33.609142408868486,548084.071741353,0.0027591623386472087,0.00013875155227786726,0.2677613556442223,2959.5714944611627,5.8227784555678714e-05,LUNG1-001_20180209_CT_2_GTV-1_mask,GTV-1_mask
1,v3.0,1.18.3,1.1.0,0.5.2,3.6.4,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [1.0, 2.0, 3...",5c66c70353901fd238191b88a2584cd8c3f645e5,3D,"(0.977, 0.977, 3.0)",...,13208.12549259126,55.600106536368784,832176.2485225748,0.004497426306875555,0.00016233377171621347,0.1889311894310745,1733.8368052732842,5.3429352136211716e-05,LUNG1-002_20180526_CT_1_GTV-1_mask,GTV-1_mask
2,v3.0,1.18.3,1.1.0,0.5.2,3.6.4,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [1.0, 2.0, 3...",9a2b123cabcc00593e4255d41cbef78f0cb63630,3D,"(0.977, 0.977, 3.0)",...,9142.646956472132,17.90900792971647,209143.44409264647,0.0026732953405101414,0.0003673562141585293,0.4029303292416708,2838.784544208129,0.00019106863067788412,LUNG1-003_20180209_CT_1_GTV-1_mask,GTV-1_mask
3,v3.0,1.18.3,1.1.0,0.5.2,3.6.4,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [1.0, 2.0, 3...",9a2b123cabcc00593e4255d41cbef78f0cb63630,3D,"(0.977, 0.977, 3.0)",...,1959.5870913486808,57.127036686639336,110707.13145169767,0.030102868247191432,0.0005756760844488658,0.08743995052119405,201.12707131279166,8.353151337486906e-05,LUNG1-003_20180209_CT_1_GTV-2_mask,GTV-2_mask
4,v3.0,1.18.3,1.1.0,0.5.2,3.6.4,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [1.0, 2.0, 3...",9a2b123cabcc00593e4255d41cbef78f0cb63630,3D,"(0.977, 0.977, 3.0)",...,12522.929147667484,21.910557699052966,283996.4951479013,0.0018352259335414646,0.00020430382406090388,0.1809366847587144,2164.128732620688,0.00013862188235453052,LUNG1-003_20180209_CT_1_GTV-3_mask,GTV-3_mask


In [15]:
#Erite features to excel file
writer = pd.ExcelWriter('features.xlsx')
features.to_excel(writer,'Sheet1')
writer.save()